In [ ]:
import sys
sys.path.append('../30_data_tools/')
sys.path.append('../60_gebastel/Musterueberlagerung/pattern_effects/')

In [ ]:
import numpy as np
from PIL import Image, ImageChops, ImageOps
from helper import load_dotenv
from trapezoidal_distortion import distort_trapezoidal
import plotly.express as px
import math
from scipy import ndimage

In [ ]:
def get_fft( input_img ):
    ft = np.fft.ifftshift(np.array(input_img))
    ft = np.fft.fft2(ft)
    ft = np.fft.fftshift(ft)
    
    return ft

In [ ]:
def limit_frequencies( fft, inner_limit=None, outer_limit=None ):
    center = (fft.shape[1] / 2, fft.shape[0] / 2)
    for y in range(fft.shape[0]):
        for x in range(fft.shape[1]):
            r = math.sqrt( abs(center[0] - x) ** 2 + abs(center[1] - y) ** 2 )
            
            if outer_limit is not None and r > outer_limit:
                fft[y,x] = 0
    
            if inner_limit is not None and r < inner_limit:
                fft[y,x] = 0

    return fft

In [ ]:
Image.MAX_IMAGE_PIXELS = None

In [ ]:
dotenv = load_dotenv()

In [ ]:
x = np.arange(-1000, 1001, 1)
X, Y = np.meshgrid(x, x)
wavelength = 20
grating = np.sin(2 * np.pi * X / wavelength) * 0.5 + 0.5

angle = np.pi / 180 * 90
grating_rotated = np.sin(
    2*np.pi*(X*np.cos(angle) + Y*np.sin(angle)) / wavelength
) * 0.5 + 0.5

In [ ]:
grating.min()

In [ ]:
grating_img = Image.fromarray((((grating * grating_rotated) * 255).round()).astype('uint8'))
#grating_distorted = grating_img.rotate(10, fillcolor="white")
grating_distorted = distort_trapezoidal( grating_img, (0.0,0.75), "top" )
#grating_distorted = grating_img.resize(( round(grating_img.size[0] * 1.2), grating_img.size[1] ))

In [ ]:
grating_img

In [ ]:
img = Image.open(dotenv['PATTERN_DIR'] / 'plain_4.LINE.tif').crop((0,0,1000,1000))
img_line = ImageOps.invert(img).convert('L')

In [ ]:
img = Image.open(dotenv['PATTERN_DIR'] / 'plain_4.CIRCLE.tif').crop((0,0,1000,1000))
img_circle = ImageOps.invert(img).convert('L')

In [ ]:
multiplied = ImageChops.multiply( img_line, distort_trapezoidal( img_circle, (0.0,0.5), "top" ) )

In [ ]:
fft = get_fft( ImageChops.multiply( img_line, distort_trapezoidal( img_line, (0.0,0.5), "top" ) ) )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)

In [ ]:
fft = get_fft( ImageChops.multiply( img_circle, distort_trapezoidal( img_circle, (0.0,0.5), "top" ) ) )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)

In [ ]:
fft = get_fft( ImageChops.multiply( img_line, distort_trapezoidal( img_circle, (0.0,0.5), "top" ) ) )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)

In [ ]:
fft = get_fft( ImageChops.multiply( img_circle, distort_trapezoidal( img_line, (0.0,0.5), "top" ) ) )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)

In [ ]:
fft = get_fft( img_circle )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)

In [ ]:
fft = get_fft( ImageChops.multiply( img_line, img_line.rotate(10, fillcolor="white") ) )

px.imshow(
    ndimage.uniform_filter(
        np.abs(fft),
        size=5
    ),
    color_continuous_scale=['white','black']
)